In [6]:
import tensorflow as tf

from keras.layers import Conv2D, DepthwiseConv2D, AveragePooling2D, BatchNormalization, Input, Dense, Flatten, ZeroPadding2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from keras.layers.advanced_activations import ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Add
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

In [7]:
# Config variables
# Data images are not of the same size
INPUT_SIZE = [224, 224]
BATCH = 32

In [8]:
# No data augmentation
train_val_image_data_generator = ImageDataGenerator(rescale=1.0/255,
                                            rotation_range=20,
                                            zoom_range=0.2,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode="nearest",
                                            validation_split=0.2
                                            )

train_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     classes=['with_mask','without_mask'],
                                                     subset='training',
                                                     shuffle=True
                                                     )
validate_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     classes=['with_mask','without_mask'],
                                                     subset='validation',
                                                     shuffle=True
                                                     )

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [9]:
def identity_block(x, filters):
   
    f1 = filters

    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1),  padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    return x

def conv_block(x, s, filters):
    x_skip = x
    f1= filters

    x = Conv2D(f1, kernel_size=(3, 3), strides=(s, s), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    x_skip = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)

    x = Add()([x, x_skip])
    x = ReLU()(x)

    return x

input = Input(shape=INPUT_SIZE + [3])
x = ZeroPadding2D(padding=(3, 3))(input)

x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((3, 3), strides=(2, 2),padding='same')(x)

x = identity_block(x, filters=(64))
x = identity_block(x, filters=(64))


# 3th stage

x = conv_block(x, s=2, filters=(128))
x = identity_block(x, filters=(128))


# 4th stage

x = conv_block(x, s=2, filters=(256))
x = identity_block(x, filters=(256))


# 5th stage

x = conv_block(x, s=2, filters=(512))
x = identity_block(x, filters=(512))

x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=2, activation='softmax')(x)

resnet = Model(inputs=input, outputs=x, name = 'Resnet')
resnet.summary()


Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 112, 112, 64) 256         conv2d_20[0][0]                  
_____________________________________________________________________________________________

In [10]:

resnet.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor="accuracy", patience=20, verbose=1, mode="auto")

resnet_r = resnet.fit( train_data, validation_data=validate_data,epochs=25, steps_per_epoch=len(train_data),validation_steps=len(validate_data),callbacks=[es])

Epoch 1/25
189/189 [==============================] - 74s 377ms/step - loss: 0.4616 - accuracy: 0.7792 - val_loss: 0.7682 - val_accuracy: 0.5662
Epoch 2/25
189/189 [==============================] - 64s 337ms/step - loss: 0.3202 - accuracy: 0.8600 - val_loss: 0.3292 - val_accuracy: 0.8530
Epoch 3/25
189/189 [==============================] - 61s 323ms/step - loss: 0.2501 - accuracy: 0.9025 - val_loss: 0.3426 - val_accuracy: 0.8457
Epoch 4/25
189/189 [==============================] - 61s 323ms/step - loss: 0.2187 - accuracy: 0.9115 - val_loss: 0.2551 - val_accuracy: 0.9020
Epoch 5/25
189/189 [==============================] - 62s 326ms/step - loss: 0.1796 - accuracy: 0.9283 - val_loss: 0.2540 - val_accuracy: 0.8947
Epoch 6/25
189/189 [==============================] - 67s 355ms/step - loss: 0.1749 - accuracy: 0.9335 - val_loss: 1.0244 - val_accuracy: 0.6742
Epoch 7/25
189/189 [==============================] - 66s 350ms/step - loss: 0.1547 - accuracy: 0.9404 - val_loss: 0.4640 - val_ac